# Kimi API 测试

In [10]:
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(
    api_key='sk-LjTqTa7tIVwLQInDmRILGchIP9X1vXG3mARWqFMbKxNRKgVn', # 在这里将 MOONSHOT_API_KEY 替换为你从 Kimi 开放平台申请的 API Key
    base_url="https://api.moonshot.cn/v1",
)
 
# moonshot.pdf 是一个示例文件, 我们支持文本文件和图片文件，对于图片文件，我们提供了 OCR 的能力
# 上传文件时，我们可以直接使用 openai 库的文件上传 API，使用标准库 pathlib 中的 Path 构造文件
# 对象，并将其传入 file 参数即可，同时将 purpose 参数设置为 file-extract；注意，目前文件上传
# 接口仅支持 file-extract 一种 purpose 值。
file_object = client.files.create(file=Path("D:/2024秋招-大四上/毕业论文/DocuAssist_flask/static/doc/word.docx"), purpose="file-extract")
 
# 获取结果
# file_content = client.files.retrieve_content(file_id=file_object.id)
# 注意，某些旧版本示例中的 retrieve_content API 在最新版本标记了 warning, 可以用下面这行代替
# （如果使用旧版本的 SDK，可以继续延用 retrieve_content API）
file_content = client.files.content(file_id=file_object.id).text
 
# 把文件内容通过系统提示词 system prompt 放进请求中
messages = [
    {
        "role": "system",
        "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。",
    },
    {
        "role": "system",
        "content": file_content, # <-- 这里，我们将抽取后的文件内容（注意是文件内容，而不是文件 ID）放置在请求中
    },
    {"role": "user", "content": "请简单介绍该文件的具体内容"},
]
 
# 然后调用 chat-completion, 获取 Kimi 的回答
completion = client.chat.completions.create(
  model="moonshot-v1-32k",
  messages=messages,
  temperature=0.3,
)
 
print(completion.choices[0].message)

ChatCompletionMessage(content='该文件是一个包含英语单词及其对应中文解释的词汇列表。它从英文字母表的 "a" 开始，列出了一系列单词，每个单词后面跟着它的词性（如名词n.、动词vt.等）、中文解释，以及部分单词的同义词或反义词。这个列表可以用于学习英语词汇，帮助读者理解每个单词的基本含义和用法。例如，"abandon" 后面跟着 "vt. 丢弃；放弃，抛弃"，意味着 "abandon" 是一个动词，意思是丢弃或放弃。整个文件包含了从 "a" 到 "b" 开头的单词，覆盖了日常生活中的常用词汇。', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [ ]:
# 添加新的用户消息到messages列表
messages.append({"role": "user", "content": "解析文件并生成故事，来教会我里面的内容"})

# 获取新的回答
completion = client.chat.completions.create(
    model="moonshot-v1-32k",
    messages=messages,
    temperature=0.3,
)

print(completion.choices[0].message.content)

# QianWen API 测试


In [11]:
import os
from pathlib import Path
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv('sk-LjTqTa7tIVwLQInDmRILGchIP9X1vXG3mARWqFMbKxNRKgVn'),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

file_object = client.files.create(file=Path("D:/2024秋招-大四上/毕业论文/DocuAssist_flask/static/doc/word.docx"), purpose="file-extract")
print(file_object.id)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable